In [ ]:
!pip install tensorflow
!pip install tensorflow mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 49.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
import pickle
from tqdm import tqdm
import cv2
from concurrent.futures import ThreadPoolExecutor

physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        try:
            tf.config.experimental.set_memory_growth(device, True)
            tf.config.set_logical_device_configuration(
                device, [tf.config.LogicalDeviceConfiguration(memory_limit=15000)]  
            )
            print(f"Configured GPU with a memory limit of 15000 MB.")
        except Exception as e:
            print(f"Error configuring GPU: {e}")
else:
    print("No GPU detected, running on CPU.")

try:
    tf.keras.mixed_precision.set_global_policy('mixed_float16')
    print("Mixed precision enabled for speedup.")
except ValueError:
    print("Mixed precision not supported, running with default precision.")

def save_progress(filepath, data):
    try:
        with open(filepath, 'wb') as f:
            pickle.dump(data, f)
        print(f"Progress saved at: {filepath}")
    except Exception as e:
        print(f"Error saving progress: {e}")

def load_progress(filepath):
    if os.path.exists(filepath):
        try:
            with open(filepath, 'rb') as f:
                return pickle.load(f)
        except Exception as e:
            print(f"Error loading progress: {e}")
    return []

def preprocess_image(image_path):
    try:
        image = cv2.imread(image_path)
        if image is None:
            print(f"Failed to read image: {image_path}")
            return np.zeros((224, 224, 3), dtype=np.float32)

        image = cv2.resize(image, (224, 224))
        image = preprocess_input(image.astype(np.float32))
        return image
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return np.zeros((224, 224, 3), dtype=np.float32)

def preprocess_images_multithreaded(frame_files, num_threads=4):
    processed_images = []
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        results = list(tqdm(executor.map(preprocess_image, frame_files), total=len(frame_files), desc="Preprocessing images"))
        processed_images.extend(results)
    return np.array(processed_images, dtype=np.float32)

def extract_spatial_features_with_checkpoint(video_dir, model, checkpoint_path, batch_size=32, save_interval=200, num_threads=4):
    features = load_progress(checkpoint_path) or []
    processed_videos = {f['video_name'] for f in features if 'video_name' in f}
    all_videos = [os.path.join(video_dir, f) for f in os.listdir(video_dir) if os.path.isdir(os.path.join(video_dir, f))]
    remaining_videos = [v for v in all_videos if v not in processed_videos]

    print(f"Total videos: {len(all_videos)}, Remaining videos: {len(remaining_videos)}")

    for video_path in tqdm(remaining_videos, desc="Extracting spatial features"):
        frame_files = [
            os.path.join(video_path, frame) for frame in os.listdir(video_path)
            if frame.lower().endswith(('.jpg', '.png', '.jpeg'))
        ]
        if not frame_files:
            print(f"No image files found in directory: {video_path}")
            continue

        frame_files.sort()

        try:
            processed_frames = preprocess_images_multithreaded(frame_files, num_threads)
            dataset = tf.data.Dataset.from_tensor_slices(processed_frames).batch(batch_size).prefetch(tf.data.AUTOTUNE)

            features_batch = model.predict(dataset, verbose=1)
            features.append({'video_name': video_path, 'features': features_batch})
        except Exception as e:
            print(f"Error extracting features for {video_path}: {e}")

        if len(features) % save_interval == 0:
            save_progress(checkpoint_path, features)

    save_progress(checkpoint_path, features)
    return features

if __name__ == "__main__":
    base_path = 'drive/MyDrive/final_dataset/cropped_frames/'
    features_dir = 'drive/MyDrive/final_dataset/features/'
    os.makedirs(features_dir, exist_ok=True)

    real_frames_dir = os.path.join(base_path, 'real/')
    fake_frames_dir = os.path.join(base_path, 'fake/')

    real_checkpoint = os.path.join(features_dir, 'spatial_features_real.pkl')
    fake_checkpoint = os.path.join(features_dir, 'spatial_features_fake.pkl')

    efficientnet = EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')

    print("Processing real videos...")
    spatial_features_real = extract_spatial_features_with_checkpoint(
        video_dir=real_frames_dir,
        model=efficientnet,
        checkpoint_path=real_checkpoint,
        batch_size=32,
        save_interval=200,
        num_threads=4
    )

    print("Processing fake videos...")
    spatial_features_fake = extract_spatial_features_with_checkpoint(
        video_dir=fake_frames_dir,
        model=efficientnet,
        checkpoint_path=fake_checkpoint,
        batch_size=32,
        save_interval=200,
        num_threads=4
    )

    print("Spatial feature extraction completed.")


Configured GPU with a memory limit of 15000 MB.
Mixed precision enabled for speedup.
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Processing real videos...
Total videos: 967, Remaining videos: 967


Preprocessing images: 100%|██████████| 48/48 [00:03<00:00, 13.79it/s]


2/2 ━━━━━━━━━━━━━━━━━━━━ 18s 8s/step


Preprocessing images: 100%|██████████| 48/48 [00:03<00:00, 14.89it/s]

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step


Preprocessing images: 100%|██████████| 16/16 [00:01<00:00, 12.15it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step



Extracting spatial features: 100%|██████████| 967/967 [19:28<00:00,  1.21s/it]

Progress saved at: drive/MyDrive/final_dataset/features/spatial_features_fake.pkl
Spatial feature extraction completed.
